In [2]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

# We import the state wise data for population of USA

In [3]:
link = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(link) 
soup = BeautifulSoup(page.text)

## we obtain all the tables in the webpage, our table of interest lies at the 4th index

In [4]:

table = soup.find_all('table')[4]

In [5]:
table


<table class="wikitable sortable" style="text-align:center">
<tbody><tr>
<th>2018<br/>rank
</th>
<th>City
</th>
<th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[c]</a></sup>
</th>
<th>2018<br/>estimate
</th>
<th>2010<br/>Census
</th>
<th>Change
</th>
<th colspan="2">2016 land area
</th>
<th colspan="2">2016 population density
</th>
<th>Location
</th></tr>
<tr>
<td>1
</td>
<td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York City">New York</a></i><sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[d]</a></sup>
</td>
<td align="left"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="450" data-file-width="900" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_New_York.svg/23px-Flag_of_New_York.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_New_York.svg/35px-Flag_of_New_York.svg.png 1.5x, //upload.wikimedia.org/w

### the dataframe rows are populated, and are named accordingly

In [6]:
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Rank", "City", "State", "del1", "del2", "del3", "Sq.Area", "del5", "population density in Sq Mi", "Population density in Km2", "Location"])
df.head()

,Rank,City,State,del1,del2,del3,Sq.Area,del5,population density in Sq Mi,Population density in Km2,Location
0,1,New York[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


In [7]:
new= df["Sq.Area"].str.split("s", n=1, expand = True)
new = new[0].str.replace(u'\xa0',u'')
df["Sq.Area"] = new.str.replace(',','')
df["Sq.Area"] = df["Sq.Area"].astype(float)
df["Radius"] = np.sqrt(df["Sq.Area"])
df.head()

,Rank,City,State,del1,del2,del3,Sq.Area,del5,population density in Sq Mi,Population density in Km2,Location,Radius
0,1,New York[d],New York,"8,398,748","8,175,133",+2.74%,301.5,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,17.363755
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,21.649480
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,15.076472
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,25.248762
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,22.750824


## The unnecessary columns like rank,sq.area etc are dropped from our dataframe and only the relevant columns are retained

In [8]:
df.drop(columns = ["Rank", "del1", "del2", "del3", "del5", "Sq.Area", "population density in Sq Mi"], inplace = True)
df.column

AttributeError: 'DataFrame' object has no attribute 'column'

In [9]:
df.head()

,City,State,Population density in Km2,Location,Radius
0,New York[d],New York,"10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,17.363755
1,Los Angeles,California,"3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,21.649480
2,Chicago,Illinois,"4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,15.076472
3,Houston[3],Texas,"1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,25.248762
4,Phoenix,Arizona,"1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,22.750824


In [10]:
df["Location"]= df["Location"].str.split("/", n = 2, expand = True)[1]
df.head()

,City,State,Population density in Km2,Location,Radius
0,New York[d],New York,"10,933/km2",﻿40.6635°N 73.9387°W﻿,17.363755
1,Los Angeles,California,"3,276/km2",﻿34.0194°N 118.4108°W﻿,21.649480
2,Chicago,Illinois,"4,600/km2",﻿41.8376°N 87.6818°W﻿,15.076472
3,Houston[3],Texas,"1,395/km2",﻿29.7866°N 95.3909°W﻿,25.248762
4,Phoenix,Arizona,"1,200/km2",﻿33.5722°N 112.0901°W﻿,22.750824


In [11]:
new = df["Location"].str.split(" ", n = 0, expand = False)
k = df.copy(deep = True)

## The latitudes and longitudes are displayed alonwith the radius of every city

In [12]:
Latitude = []
Longitude = []
for i in range(len(new)):
    Latitude.append(new[i][1][:-2])
    Longitude.append(new[i][2][:-3]) 

k["Latitude"] = Latitude
k["Longitude"] = Longitude
k["Latitude"] = k["Latitude"].str.replace(u'\ufeff',u'')
k.drop(columns = ["Location"], inplace = True)
k.head()
df = k.copy(deep = True)

In [13]:
k.head()

,City,State,Population density in Km2,Radius,Latitude,Longitude
0,New York[d],New York,"10,933/km2",17.363755,40.6635,73.9387
1,Los Angeles,California,"3,276/km2",21.649480,34.0194,118.4108
2,Chicago,Illinois,"4,600/km2",15.076472,41.8376,87.6818
3,Houston[3],Texas,"1,395/km2",25.248762,29.7866,95.3909
4,Phoenix,Arizona,"1,200/km2",22.750824,33.5722,112.0901


In [14]:
df['Longitude'] = -df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)
df['Radius'] = df['Radius']* 1000
df.head()

,City,State,Population density in Km2,Radius,Latitude,Longitude
0,New York[d],New York,"10,933/km2",17363.755354,40.6635,-73.9387
1,Los Angeles,California,"3,276/km2",21649.480363,34.0194,-118.4108
2,Chicago,Illinois,"4,600/km2",15076.471736,41.8376,-87.6818
3,Houston[3],Texas,"1,395/km2",25248.762346,29.7866,-95.3909
4,Phoenix,Arizona,"1,200/km2",22750.824161,33.5722,-112.0901


# Getting the per capita income details according to counties


In [15]:
link1 = 'https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income'
page1 = requests.get(link1) 
soup1 = BeautifulSoup(page1.text)
table = soup1.find_all('table')[2]
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df_state = pd.DataFrame(res, columns=["Rank", "Country-equivalent", "State", "Per capita income", "del2", "del3", "Population", "del5"])
df_state.head()

,Rank,Country-equivalent,State,Per capita income,del2,del3,Population,del5
0,1,New York County,New York,"$62,498","$69,659","$84,627","1,605,272","736,192"
1,2,Arlington,Virginia,"$62,018","$103,208","$139,244","214,861","94,454"
2,3,Falls Church City,Virginia,"$59,088","$120,000","$152,857","12,731","5,020"
3,4,Marin,California,"$56,791","$90,839","$117,357","254,643","102,912"
4,5,Alexandria City,Virginia,"$54,608","$85,706","$107,511","143,684","65,369"


## Dropping the unnecessary columns we get:

In [16]:
df_state.drop(columns = ['Rank','del2', 'del3', 'del5'], axis = 1, inplace = True)

In [17]:
df_state.head()

,Country-equivalent,State,Per capita income,Population
0,New York County,New York,"$62,498","1,605,272"
1,Arlington,Virginia,"$62,018","214,861"
2,Falls Church City,Virginia,"$59,088","12,731"
3,Marin,California,"$56,791","254,643"
4,Alexandria City,Virginia,"$54,608","143,684"


# Plotting of the cities

In [18]:
# create map of USA cities that we have using latitude and longitude values
map_tohood = folium.Map(location=[37.0902,-95.7129], zoom_start=3)

# add markers to map
for lat, lng, state, city in zip(df['Latitude'], df['Longitude'], df['State'], df['City']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_tohood)  
    
map_tohood

In [19]:
CLIENT_ID = 'SODMMH3ADBUMWYASIXH3L2UZBOC4NLR1ZQZ1BIBZ2KIFOV3I' # your Foursquare ID
CLIENT_SECRET = 'RFIREFN2E2OJWQXUO3KJHLGA4MKV1PBM1K2CMOJTBF3AHTBA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 20
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SODMMH3ADBUMWYASIXH3L2UZBOC4NLR1ZQZ1BIBZ2KIFOV3I
CLIENT_SECRET:RFIREFN2E2OJWQXUO3KJHLGA4MKV1PBM1K2CMOJTBF3AHTBA


## function to get nearby venues from a city

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng,radius in zip(names, latitudes, longitudes,radius):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       # print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
df_venues = getNearbyVenues(names = df['City'], latitudes = df['Latitude'],longitudes = df['Longitude'], radius = df['Radius'])
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden
1,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar
2,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum
3,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater
4,New York[d],40.6635,-73.9387,Prospect Park (Nethermead),40.660717,-73.968587,Field


## Assigning Weights to venue categories to determine whether the given venue is appropriate for our business scenario

In [28]:
k = df_venues.copy(deep = True)
weights_dict={'Movie Theater':3,'Beach':3,'Concert Hall':2.5,'Playground':3,'Coffee Shop':3.5,'Food Court':4,'Nightclub':4,'Toy / Game Store':4.5,'Theme Park Ride / Attraction':4,'Pub':4}
data = df_venues['Venue Category']
allVenues = data[:]
type(allVenues)

pandas.core.series.Series

In [29]:
weights = []
for i in allVenues:
    if i in weights_dict.keys():
        weights.append(weights_dict[i])
    else :
        weights.append(0)
df_venues['weights'] = weights;
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden,0.0
1,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar,0.0
2,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum,0.0
3,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater,0.0
4,New York[d],40.6635,-73.9387,Prospect Park (Nethermead),40.660717,-73.968587,Field,0.0


## we have to extract the list of all venues where the venue category matches our requirement.

### Here we are selecting and retaining the values where the weights are over a certain threshold i.e 1

In [30]:
df_venues.drop(df_venues[df_venues.weights < 1.0].index, inplace=True)
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
21,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,34.027115,-118.387637,Coffee Shop,3.5
29,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,33.980027,-118.408020,Coffee Shop,3.5
55,Chicago,41.8376,-87.6818,Sawada Coffee,41.883730,-87.648726,Coffee Shop,3.5
75,Houston[3],29.7866,-95.3909,White Oak Music Hall,29.785994,-95.367173,Concert Hall,2.5
88,Phoenix,33.5722,-112.0901,Dutch Bros. Coffee,33.538051,-112.055962,Coffee Shop,3.5


## Now we are finding the average of the weights citywise.

### This will help us todetermine which city would be the most suited for our purposes

In [31]:
citywise_venues_weights = df_venues[['City','weights']].copy()
citywise_venues_weights_means = citywise_venues_weights.groupby(['City']).mean()
citywise_venues_weights_means = citywise_venues_weights_means.reset_index(drop=False)
citywise_venues_weights_means.head()

,City,weights
0,Abilene,3.5
1,Alexandria[m],3.5
2,Allen,2.5
3,Amarillo,3.5
4,Anaheim,3.5


# Merging the tables so that we obtain the city,weights as well as the population figures earlier obtained

In [32]:
city_selection = pd.merge(df, citywise_venues_weights_means, on='City')
city_selection = city_selection[['City','Population density in Km2','weights']].copy()
city_selection.head()

,City,Population density in Km2,weights
0,Los Angeles,"3,276/km2",3.5
1,Chicago,"4,600/km2",3.5
2,Houston[3],"1,395/km2",2.5
3,Phoenix,"1,200/km2",3.5
4,Philadelphia[e],"4,511/km2",3.5


In [33]:
k = city_selection.copy(deep = True)
k['Population density in Km2'] = k['Population density in Km2'].str.split("/", n = 0, expand = True)
k['Population density in Km2'] = k['Population density in Km2'].str.replace(',','')
k['Population density in Km2'] = k['Population density in Km2'].astype(float)
city_selection = k.copy(deep = True)
city_selection.head()

,City,Population density in Km2,weights
0,Los Angeles,3276.0,3.5
1,Chicago,4600.0,3.5
2,Houston[3],1395.0,2.5
3,Phoenix,1200.0,3.5
4,Philadelphia[e],4511.0,3.5


# Normalizing the dataframe

In [34]:
from sklearn import preprocessing
column_names_to_normalize = ['Population density in Km2', 'weights']
x = city_selection[column_names_to_normalize].values #returns a numpy array
x

array([[3.27600000e+03, 3.50000000e+00],
       [4.60000000e+03, 3.50000000e+00],
       [1.39500000e+03, 2.50000000e+00],
       [1.20000000e+03, 3.50000000e+00],
       [4.51100000e+03, 3.50000000e+00],
       [1.67000000e+03, 3.75000000e+00],
       [4.55000000e+02, 3.00000000e+00],
       [1.52000000e+03, 3.50000000e+00],
       [9.14000000e+02, 2.50000000e+00],
       [3.24500000e+03, 3.16666667e+00],
       [1.74600000e+03, 3.50000000e+00],
       [4.30400000e+03, 3.50000000e+00],
       [5.38100000e+03, 3.00000000e+00],
       [1.03000000e+03, 3.25000000e+00],
       [5.36000000e+02, 2.50000000e+00],
       [1.85100000e+03, 3.50000000e+00],
       [7.94000000e+02, 3.33333333e+00],
       [4.07000000e+02, 3.00000000e+00],
       [9.03000000e+02, 3.50000000e+00],
       [2.93400000e+03, 3.37500000e+00],
       [2.38800000e+03, 3.33333333e+00],
       [8.88000000e+02, 3.50000000e+00],
       [1.76200000e+03, 3.50000000e+00],
       [1.35700000e+03, 3.25000000e+00],
       [1.953000

In [35]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
city_selection[column_names_to_normalize] = pd.DataFrame(x_scaled)
city_selection.head()

,City,Population density in Km2,weights
0,Los Angeles,0.470174,0.5
1,Chicago,0.664224,0.5
2,Houston[3],0.194489,0.0
3,Phoenix,0.165909,0.5
4,Philadelphia[e],0.651180,0.5


## Calculate the sum of normalized values city-wise

In [36]:
city_selection['sum'] = city_selection['Population density in Km2'] + city_selection['weights']
row_num = city_selection['sum'].argmax()
city_name = city_selection['City'].iloc[row_num]
city_name

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  from ipykernel import kernelapp as app


'Jersey City'

## Thus we evaluated that Jersey City would be the best city for setting up our arcade store.

In [41]:
row = df.loc[df['City']== city_name].index[0]
state_name = df['State'].iloc[row]
state_name

'New Jersey'

## The per capita income of the state should be more than USD 45000

In [43]:
p_row = df_state.loc[df_state['State'] == state_name].index[0]
per_capital_income = df_state['Per capita income'].iloc[p_row]
print("Per capita income of New Jersey is :", per_capital_income)

Per capita income of New Jersey is : $50,349


### As Jersey city is an ideal choice for setting up an arcade, we should examine which location is best-suited for our interests

In [44]:
lat_newJersey = df['Latitude'].iloc[row]
long_newJersey = df['Longitude'].iloc[row]
print(lat_newJersey, long_newJersey)

40.7114 -74.0648


In [46]:
# Getting the venues of New Jersey using four square API 

def getNearbyVenues1(name, latitudes, longitudes, radius):
    
    LIMIT = 150       
        # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudes, 
            longitudes, 
            radius, 
            LIMIT)
            
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
   # print(results)
    venues_list=[]
    venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category']
    return(nearby_venues)


new_jersey_venues = getNearbyVenues1(name = 'Jersey City', latitudes = lat_newJersey ,longitudes = long_newJersey, radius = 2500)
new_jersey_venues.head(1)

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jersey City,38.3539,-121.9728,The Grind Shop,40.71167,-74.062872,Coffee Shop


In [47]:
venues_in_newjersey = new_jersey_venues.copy(deep = True)
venues_in_newjersey.shape

(100, 7)

### Assigning random weights to each category, according to the type of Venue we think should be better for our business 

In [48]:
k = new_jersey_venues.copy(deep = True)
new_weightage_dict= {'Coffee Shop' : 3, 
'Caribbean Restaurant':3,
'Distillery':2,
'Fish & Chips Shop':3,
'Science Museum':3,
'Latin American Restaurant':4,
'Restaurant':5,
'State / Provincial Park':1,
'Diner':1,
'Supermarket':1,
'Bar':1,
'Jazz Club':1,
'Golf Course':3,
'Park':2,
'Cajun / Creole Restaurant':2,
'Bakery':2,
'Go Kart Track':3,
'Taco Place':3,
'Hot Dog Joint':2,
'Food Truck':3,
'Beer Garden':3,
'Boutique':4,
'Café':5,
'Bagel Shop':1,
'Record Shop':1,
'Bakery':1,
'Pizza Place':1,
'Ramen Restaurant':1,
'Wine Bar':3,
'Middle Eastern Restaurant':2,
'French Restaurant':2,
'Theater':2,
'Lounge':3,
'Wine Shop':3,
'Cocktail Bar':2,
'New American Restaurant':3,
'Residential Building (Apartment / Condo)':3,
'Pool':4,
'Burger Joint':5,
'Cheese Shop':1,
'Coffee Shop':1,
'Bagel Shop':1,
'Vietnamese Restaurant':1,
'Portuguese Restaurant':1,
'Ice Cream Shop':3,
'Italian Restaurant':2,
'Gym':2,
'Farmers Market':2,
'Bar':3,
'Pizza Place':3,
'Bakery':2,
'Bookstore':3,
'Bar':3,
'Farmers Market':4,
'Asian Restaurant':5,
'Tea Room':1,
'Donut Shop':1,
'Historic Site':1,
'Gym / Fitness Center':1,
'Café':1,
'Mexican Restaurant':3,
'Plaza':2,
'Gay Bar':2,
'Bar':3,
'College Administrative Building':3,
'Mexican Restaurant':2,
'Bakery':3,
'American Restaurant':3,
'American Restaurant':4,
'American Restaurant':5,
'Café':1,
'New American Restaurant':1,
'Chocolate Shop':1,
'Gym':1,
'Grocery Store':1,
'Middle Eastern Restaurant':3,
'American Restaurant':2,
'Frozen Yogurt Shop':2,
'Japanese Restaurant':2,
'Bar':3,
'Liquor Store':3,
'Ice Cream Shop':2,
'Fish Market':3,
'Indie Movie Theater':3,
'Grocery Store':4,
'Modern European Restaurant':5,
'American Restaurant':1,
'Poke Place':1,
'Ramen Restaurant':1,
'Diner':1,
'Brewery':1,
'Burger Joint':3,
'Burger Joint':2,
'Café':2,
'Fried Chicken Joint':2,
'Beer Garden':3,
'Gym / Fitness Center':3,
'Vietnamese Restaurant':2,
'Italian Restaurant':3,
'Pet Store':3}

In [49]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# create map of the venues that we have using latitude and longitudes
venues_map = folium.Map(location=[lat_newJersey, long_newJersey], zoom_start=15) # generate map centred around Jersey city


# add Jersey City as a red circle mark
folium.features.CircleMarker(
    [lat_newJersey, long_newJersey],
    radius=10,
    popup='Jersey city',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

In [50]:
for lat, lng, label in zip(venues_in_newjersey['Venue Latitude'], venues_in_newjersey['Venue Longitude'], venues_in_newjersey['Venue']):
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html = False).add_to(venues_map)
venues_map

In [51]:

allVenuesinCity1 = k['Venue Category']

f_weights1 = []
for i in allVenuesinCity1:
    if i in new_weightage_dict.keys():
        f_weights1.append(new_weightage_dict[i])
    else :
        f_weights1.append(0)
k['weights'] = f_weights1;
k.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,Jersey City,38.3539,-121.9728,The Grind Shop,40.711670,-74.062872,Coffee Shop,1
1,Jersey City,38.3539,-121.9728,Harry’s Daughter,40.710904,-74.062071,Caribbean Restaurant,3
2,Jersey City,38.3539,-121.9728,Corgi Spirits at The Jersey City Distillery,40.708304,-74.064803,Distillery,2
3,Jersey City,38.3539,-121.9728,Hooked JC,40.714709,-74.067009,Fish & Chips Shop,3
4,Jersey City,38.3539,-121.9728,Liberty Science Center,40.707881,-74.055121,Science Museum,3


In [52]:

newframe = k[['City','Venue Category','weights']].copy()
newframe = k.groupby(['Venue Category']).mean()
newframe.drop(columns = ["Latitude", "Longitude"], inplace = True)
newframe

,Venue Latitude,Venue Longitude,weights
Venue Category,,,
American Restaurant,40.714692,-74.041587,1
Asian Restaurant,40.721099,-74.044339,5
Australian Restaurant,40.717187,-74.044216,0
Bagel Shop,40.722990,-74.058068,1
Bakery,40.721297,-74.048836,3
Bar,40.718000,-74.056019,3
Beer Garden,40.715149,-74.046633,3
Bookstore,40.719984,-74.043205,3
Boutique,40.717606,-74.044299,4


# Clustering using K-Means, We divide the city into three clusters for easier evaluation of the exact location on which we should invest

In [53]:
# Cluster them using K means algorithm 
from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
#Standardize
clmns = ['weights','Venue Latitude', 'Venue Longitude']
df_tr_std = stats.zscore(newframe[clmns])
#Cluster the data
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_tr_std)
labels = kmeans.labels_
newframe['clusters'] = labels
#Add the column into our list
clmns.extend(['clusters'])
#Lets analyze the clusters
kframe = newframe[clmns].groupby(['Venue Category']).mean()
kframe = kframe.reset_index(drop = False)
kframe.head()

,Venue Category,weights,Venue Latitude,Venue Longitude,clusters
0,American Restaurant,1,40.714692,-74.041587,1
1,Asian Restaurant,5,40.721099,-74.044339,2
2,Australian Restaurant,0,40.717187,-74.044216,1
3,Bagel Shop,1,40.722990,-74.058068,1
4,Bakery,3,40.721297,-74.048836,2


In [54]:
finalWeight = kframe.groupby(['clusters']).mean()
finalWeight
#Finalweight is a dataframe that is used to view the clusters 

,weights,Venue Latitude,Venue Longitude
clusters,,,
0,2.538462,40.709272,-74.065527
1,1.352941,40.720288,-74.047986
2,3.363636,40.719764,-74.046841


## The maximum weight is 3.36 and the correspong location co-ordinates are picked up

In [60]:
w=finalWeight.loc[finalWeight['weights'].idxmax()]
w

weights             3.363636
Venue Latitude     40.719764
Venue Longitude   -74.046841
Name: 2, dtype: float64

* the lat and long details of the venue is obtained

In [1]:
lat1 = 40.720102
long1 = -74.048121

In [62]:
final_map = folium.Map(location=[lat1, long1], zoom_start=15) # generate map centred around Jersey city


# add prefered location in the City as a green circle mark
folium.features.CircleMarker(
    [lat1, long1],
    radius=50,
    popup='Gaming arcade can be installed within this circle',
    fill=True,
    color='green',
    fill_color='green',
    fill_opacity=0.6
    ).add_to(final_map)
final_map

## Thus we obtain the most suited location in Jersey City for setting up our own arcade

### The given model can be improved by enhancing the radius and increasing the number of clusters

### But the limitation is due to the fact that we currently do not use the paid version of the Foursquare API.